In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline

from feature_engine.imputation import CategoricalImputer, MeanMedianImputer
from feature_engine.encoding import OrdinalEncoder, OneHotEncoder
from feature_engine.wrappers import SklearnTransformerWrapper
from sklearn.preprocessing import StandardScaler, LabelBinarizer

from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import VotingClassifier, RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

from sklearn.preprocessing import FunctionTransformer

from sklearn import set_config
set_config(display='diagram')
# ------------------------------------------
import nltk
import sys
import random

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.compose import ColumnTransformer

sys.path.append('..')
import utils
from sklearn.preprocessing import FunctionTransformer

## PROPIAS
from utils import RemoveStopWords
from utils import FeatureExtractionTwitts
from utils import LemmantizerTransformer
from utils import Vectorizer
from utils import ColumnSelectedTransformer

https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.FunctionTransformer.html

In [2]:
df = pd.read_csv('training_tweets.csv', index_col = 0)
df;

In [3]:
df.content.to_csv('raw_tweets.csv')

In [4]:
groups_classes = {
                    'happiness' : 'positiva',
                    'surprise'  : 'positiva',
                    'love'      : 'positiva',
                    'fun'       : 'positiva',
                    'relief'    : 'positiva',
                    'enthusiasm': 'positiva',
                    'worry'     : 'negativa',
                    'hate'      : 'negativa',
                    'sadness'   : 'negativa',
                    'empty'     : 'negativa',
                    'boredom'   : 'negativa',
                    'anger'     : 'negativa',
                    'neutral'   : 'neutral'
                }
neutral_class_name = 'neutral'
target_var_name = 'sentiment'
target_mapping = [('positiva',1),('negativa',0)]
chars_to_replace = [
                        ('ï¿½',''),
                        ('&quot;',''),
                        ('&lt;3','<3'), 
                        ('&lt;/3','</3'), 
                        ('&amp;','&'),
                        ('&gt;','>'),
                        ('&lt;','<')
                    ]


In [5]:
from sklearn.compose import TransformedTargetRegressor
df_preprocess = Pipeline(steps=[
                ('regroup_classes', 
                    FunctionTransformer(
                        utils.multi_class_remapping, 
                        kw_args={
                            'group_classes': groups_classes,
                            'var_name': target_var_name,
                            'neutral_class': neutral_class_name,
                            'random_state': 42
                            })),
                ('encoding', FunctionTransformer(
                        utils.target_encoding,
                        kw_args={
                            'column_to_encode': 'sentiment_remapped', 'mapping':target_mapping})),
])

df = df_preprocess.fit_transform(df) #
df
# Este debiese ser la primera transformación, no se si serializar esto o no pero habría que indicar que es necesario aplicarlo a df para poder continuar.

,content,sentiment,sentiment_remapped
34723,Happy Mama's day to all mothers,love,1
17493,@LysdelTellez I am lost. Please help me find a...,worry,0
20198,"@BoomKatt yes yes I AM, networking whore to th...",happiness,1
6855,you@snapplynn Wish that would have been your t...,neutral,0
5924,now i am doing the MicroEconomics project iha...,worry,0
...,...,...,...
16469,I do not want to work tomorrow!,sadness,0
36006,@KandyBee we shuld do a dance like that its s...,fun,1
22647,"Photo: Got my prints a few days ago, ready for...",happiness,1
21478,@tove_liden Thanks for the follow Tove!,fun,1


In [6]:
# Todos los pipelines siguientes deberían vivir en un solo pipeline y al final aplicar fit_transform. Se debe serializar ese ultimo pipeline

# hiperparametros posibles:
# usar o no STOPWORDS
# stemmers = 'ps' | 'wnl' | 'sno'
stemmer = 'sno'
wether_or_not_sw = True
preprocessing = Pipeline(steps=[
                ('rc', FunctionTransformer(utils.remove_chars, kw_args={'var_name': 'content', 'char_list': chars_to_replace})),
                ('ra', FunctionTransformer(utils.remove_arrobas, kw_args= {'var_name':'content_remchars'})),
                ('rl', FunctionTransformer(utils.remove_links, kw_args={'var_name': 'content_remchars_remarroba'})),
                ('rsw', RemoveStopWords(text_columns = ['content_remchars_remarroba_remlinks'], bool_trans = wether_or_not_sw)),
                ('lt', LemmantizerTransformer(text_columns = [f"content_remchars_remarroba_remlinks{'_sw' if wether_or_not_sw else ''}"], stemmer=stemmer)),
                
])

# hiperparametros posibles:
# min_df = 1,2,3...
# max_df = 0.7 - 1.0
# vect_type = 'count' | 'tfid'

# Si se quiere ver los resultados de las limpiezas de los textos, comentar la linea de ColumnSelectedTransformer y luego ver el dataframe. También se pueden ver los reportes generados en csv y txt en la carpeta raíz.

feature_extraction = Pipeline(steps=[
                ('fet_1', FeatureExtractionTwitts(text_column="content",features_to_extract=["arrobas_count", "hashtag_count", "is_reply", "is_rt", "twitt_length"])),
                ('fet_2', FeatureExtractionTwitts(text_column=f"content_remchars_remarroba_remlinks{'_sw' if wether_or_not_sw else ''}_stemmer",features_to_extract = ["subjectivity", "polarity"])),
                ('ctr', Vectorizer(vect_type='tfid', text_column=f"content_remchars_remarroba_remlinks{'_sw' if wether_or_not_sw else ''}_stemmer", min_df=15, max_df=0.7, ngram_range=(1,3))),
                ('ds', ColumnSelectedTransformer(vars_prefix='var_'))
                
])

# joblib.load


In [7]:
from sklearn.ensemble import StackingClassifier
from sklearn.naive_bayes import MultinomialNB

mnnb = MultinomialNB()
clf1 = GaussianNB()
clf2 = KNeighborsClassifier()
clf3 = SVC(random_state=42, probability=True)
clf4 = GradientBoostingClassifier(random_state=42)

meta0 = LogisticRegression(random_state=42) # funciona particularmente bien stackeando xgboost
meta1 = RandomForestClassifier(random_state=42, n_estimators=100, max_depth=10)
meta2 = GradientBoostingClassifier(random_state=42)


# sería ideal probar:
# GradientBoosting (Camilo)
# HistGradientBoostingClassifier corre más rapido >10000 datos
# VotingClassifier (Braulio)
# StackingClassifier (Camilo)
# Red Neuronal con Keras (KerasClassifier) (Thomas)
# XGBoost, LightGBM, CatBoost (probar)

estimators = [('gnb', clf1), ('gbc', clf4)]

# final estimator: metaestimator
st_1 = StackingClassifier(estimators=estimators,
                        final_estimator=meta2,
                        cv=5)

params = {
    'preprocessing__lt__stemmer': ['sno', 'ps', 'wnl'],
    'feature_extraction__ctr__vect_type': ['tfid'],
    'feature_extraction__ctr__min_df': [5, 10, 15],
    'model__n_estimators': [50, 100, 500],
    'model__max_depth': [5, 10, 50]
}

In [8]:
def train_function(pipe, X_train, X_test, y_train, y_test):
    pipe.fit(X_train, y_train)
    y_pred_train = pipe.predict(X_train)
    y_pred = pipe.predict(X_test)
    print(classification_report(y_train, y_pred_train, digits=4))
    print(classification_report(y_test, y_pred, digits=4))
    return pipe


In [9]:
X = df[['content']]
y = df.sentiment_remapped
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.2, random_state=42)

pipe = Pipeline(steps=[
    ('preprocessing', preprocessing),
    ('feature_extraction', feature_extraction),
    ('model', meta2)
])

# train_function(pipe, X_train, X_test, y_train, y_test)

search = GridSearchCV(pipe, params, cv=5, scoring='accuracy', n_jobs=-1)
search.fit(X_train, y_train)
search.best_params_, search.best_score_


In [ ]:
pipe['feature_extraction']['ds'].vars_prefix

In [ ]:
# plt.figure(figsize=(10, 8))
# plt.hist(X_tr['var_subjectivity'], bins=50, fc=(1, 0, 0, 0.5))
# plt.hist(X_tr['var_polarity'], bins=50, fc=(0, 0, 1, 0.5))
# plt.title('Sentiment analysis')
# plt.xlabel('Polarity')
# plt.ylabel('Subjectivity')
# plt.show()

# plt.figure(figsize=(10, 8))
# plt.hist(X_tr['var_twit_length'], bins=30, fc=(1, 0, 0, 0.5))
# plt.title('Twits length')
# plt.show()

